# CS549 Machine Learning 
# Assignment 3: Support vector machine (SVM) model

### Total: 10 points

In this assignment, we will build a "toy" SVM model using a mini dataset step by step.

Your goal is to run all the cells below one by one from top to bottom. Before you run some task cells, you need to complete the missing lines (notified by "= None" in Python) in them. 

For each **task** cell that requires your completion, you can run the **evaluation** cell right after it to check if your answer correct.
The output of the evaluation cell should be the same as the "expected output" provided. (Some mismatch in the last digit of floating numbers is tolerable)

---
# Install dependencies

**quadprog** is a Python package for solving quadratic programming problems. You can install it using the following command:
```
pip install quadprog
```
Note: Windows users may need to install Visual C++ 14 first (https://visualstudio.microsoft.com/visual-cpp-build-tools/).

In [2]:
!pip install quadprog

     |████████████████████████████████| 131 kB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for quadprog: filename=quadprog-0.1.9-cp37-cp37m-linux_x86_64.whl size=301657 sha256=d6a558a6a2a0c716869dcaa18d76bacfb1bae661da2453a8fb29a6e8898f5ecb
  Stored in directory: /root/.cache/pip/wheels/69/38/c4/c986155f8609484c560a1f40cedc012ec26b6d94a14ae2aab3
Successfully built quadprog


In [3]:
import quadprog
import numpy as np


# The helper function. Dot not change it
def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = -G.T
        qp_b = -h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

# Toy data
X = np.array([
    [0, 0],
    [2, 0],
    [0, 2],
    [3, 3],
    [4, 4]
])
Y = np.array([-1, -1, -1, 1, 1])

---

# Task
We want to build an SVM model on the toy dataset: 

\begin{equation}
    x^{(1)} = (0,0),\ y^{(1)}=-1\\
    x^{(2)} = (2,0),\ y^{(2)}=-1\\
    x^{(3)} = (0,2),\ y^{(3)}=-1\\
    x^{(4)} = (3,3),\ y^{(4)}=1\\
    x^{(5)} = (4,4),\ y^{(5)}=1\\
\end{equation}

We need to solve the quadratic programming (QP) problem as the following form:

\begin{equation}
    \min_{\alpha}\big( \frac{1}{2}\alpha^{T}Q\alpha - (\textbf{1})^{T}\alpha \big) \\
    \text{subject to: } y^{T}\alpha=0,\ \alpha\geq 0
\end{equation}

The quadprog package by defaualt solves the QP as this form:

\begin{equation}
    \min_{x}\big( \frac{1}{2}x^{T}Px + q^{T}x \big) \\
    \text{subject to: } Gx\leq h,\ Ax = b
\end{equation}

Therefore, in order to use quadprog, we need to establish the responding relationships between variables: 
$P=Q$, $q = -(\textbf{1})^{T}$, $G = -(\textbf{1})^{T}$, $h=(\textbf{0})^{T}$, $A=y^T$, $b=(\textbf{0})^{T}$



---
## Task 1: Compute matrix $Q$

**3 points**

First, we need to use $x^{(i)}$ and $y^{(i)}$ to compute matrix $Q$:

\begin{equation}
    Q = \begin{bmatrix}
    y^{(1)}y^{(1)}x^{(1)T}x^{(1)} & y^{(1)}y^{(2)}x^{(1)T}x^{(2)} & \dots & y^{(1)}y^{(5)}x^{(1)T}x^{(5)} \\
    y^{(2)}y^{(1)}x^{(2)T}x^{(1)} & y^{(2)}y^{(2)}x^{(2)T}x^{(2)} & \dots & y^{(2)}y^{(5)}x^{(2)T}x^{(5)} \\
    \vdots & \vdots & \ddots & \vdots \\
    y^{(5)}y^{(1)}x^{(5)T}x^{(1)} & y^{(5)}y^{(2)}x^{(5)T}x^{(2)} & \dots & y^{(5)}y^{(5)}x^{(5)T}x^{(5)} \\
    \end{bmatrix}
\end{equation}


In [13]:
Q = np.zeros((5, 5))

### START YOUR CODE ###
for i in range(Q.shape[0]):
    for j in range(Q.shape[1]):
        # Use the ith and jth examples in X and Y to compute Q_ij
        Q[i, j] = Y[i]*Y[j]*(X[i] @ X[j])
### END YOUR CODE ###

print('Q = ', Q)

Q =  [[ 0.  0.  0.  0.  0.]
 [ 0.  4.  0. -6. -8.]
 [ 0.  0.  4. -6. -8.]
 [ 0. -6. -6. 18. 24.]
 [ 0. -8. -8. 24. 32.]]


### Expected output
&nbsp;|&nbsp;
--|--
**Q =**|[[ 0.  0.  0.  0.  0.] <br>[ 0.  4.  0. -6. -8.] <br> [ 0.  0.  4. -6. -8.] <br> [ 0. -6. -6. 18. 24.] <br> [ 0. -8. -8. 24. 32.]]

---
## Task 2: Computer other variables
**3 points**

Use the folumas: $P=Q$, $q = -(\textbf{1})^{T}$, $G = -(\textbf{1})^{T}$, $h=(\textbf{0})^{T}$, $A=y^T$, $b=(\textbf{0})^{T}$

In [19]:
### START YOUR CODE ###
P = Q + np.eye(5)*1e-5 # To solve the non-positive finite issue

# Hint: Use np.ones(), q is of length 5
q =  -1*np.ones(5)

# Hint: G is a matrix whose diagnal elements are 1s, and other elements are 0s. Use np.eye()
G = -1*np.eye(5)

# Hint: h is of length 5, with all zeros; Use np.zeros()
h = np.zeros(5)

A = Y.reshape((1,5))

# Hint: b is of length 1, with zero value; Use np.zeros()
b = np.zeros(1)

### END YOUR CODE ###

print('q = ', q)
print('G = ', G)
print('h = ', h)
print('b = ', b)

q =  [-1. -1. -1. -1. -1.]
G =  [[-1. -0. -0. -0. -0.]
 [-0. -1. -0. -0. -0.]
 [-0. -0. -1. -0. -0.]
 [-0. -0. -0. -1. -0.]
 [-0. -0. -0. -0. -1.]]
h =  [0. 0. 0. 0. 0.]
b =  [0.]


### Expected output
&nbsp;|&nbsp;
--|--
**q =**|  [-1. -1. -1. -1. -1.]
**G =**|[[-1. -0. -0. -0. -0.]<br> [-0. -1. -0. -0. -0.]<br> [-0. -0. -1. -0. -0.]<br> [-0. -0. -0. -1. -0.]<br> [-0. -0. -0. -0. -1.]]
**h =**|  [0. 0. 0. 0. 0.]
**b =**|  [0.]

---

## Task 3: Call quadprog
**1 point**

In [20]:
### START YOUR CODE ###

# Hint: Call quadprog_solve_qp() with the correct arguments
solution = quadprog_solve_qp(P,q,G,h,A,b)

### END YOUR CODE ###

print('solution = ', solution)
print('The support vectors are: ', X[solution > 0, ])

solution =  [0.         0.12499977 0.12499977 0.24999953 0.        ]
The support vectors are:  [[2 0]
 [0 2]
 [3 3]]


### Expected output
&nbsp;|&nbsp;
--|--
**solution $\approx$**|[0. 0.125 0.125 0.25 0]
**The support vectors are:** | [[2 0]<br> [0 2]<br> [3 3]]<br>

---
## Task 4: Solve the decision boundary
**3 points**

Use the support vectors to solve the $w$ and $b$ in the decision boundary $w^Tx+b=0$. Use the property that a support vector $x^{(k)}$ must satistify $y^{(k)}(w^Tx^{(k)}+b) = 1$. You can solve it with a paper and pen by listing linear equations.

In [37]:
### START YOUR ANSWERS ###
# if we solve using pen and paper we will get 3 equations
# Equation 1 -> 0.125*(2*w1+0*w2+b)=1 
# Equation 2 -> 0.125*(0*w1+2*w2+b)=1 
# Equation 3 -> 0.25*(3*w1+3*w2+b)=1 
# Now 3 unknowns and We have 3 equations, so we will solve and get those values
# Written under it.
w1 = -1
w2 = -1
b = 10
w=[w1,w2]
### END YOUR ANSWERS
print("w = ",w)
print('w1 = ', w1)
print('w2 = ', w2)
print('b = ', b)

w =  [-1, -1]
w1 =  -1
w2 =  -1
b =  10
